In [1]:
# SVM implementation for hand digits recognition
# This SVM use multiclass SVM model
# The whole process is to use binary SVM combined with One-to-rest strategy
# The hand digits recognition has label 0-9
# However, binary SVM can only classify data into two catogories
# Multiclass SVM is to select one specific digits label and compare to other digits
# The select digits is relabeled as +1, others as -1
# For each digit label, we use binary SVM to classify this one to rest
# Then obtain the score of the selected digits
# After obtaining all scores of the pixel data which are fitted in all 10 labels
# The highest score label is the label we need

In [2]:
import numpy as np
import pandas as pd
from quadprog_wrapper import solve_quadprog

In [3]:
# Read data from training set and test set
trainData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')

# Set number of samples needed from each digits subgroup
# Larger size means higher accuracy
# But it also takes more time training
sample_count = 200 #  <<== YOU MAY CHANGE THIS SIZE

In [4]:
def bin_svm_train(data, labels):
    '''
    Implementation of binary SVM
    This function is to use data and its label to perform binary SVM
    The data will be catogorized in binary class
    @param Data Dataset for binary SVM training
    @param Labels labels identified for each point, binary format
    
    @return Factors contribute to prediction, including weights, bias, support vectors and correspond label
    '''
    # Use polynomial kernel to obtain the feature of data matrix
    g_mat = (np.dot(data.T, data) + 1) ** 2
    
    # Obtain the row size of gram matrix, use for later computation
    n = g_mat.shape[0]
    
    # Hessian Matrix describes the local curvature of a function of many variables
    # It provides hessian matrix for solving quadprog problem
    hes = np.outer(labels, labels) * g_mat
    
    # Weights, linear coefficents for the gram matrix
    # Initial weights are all 0
    w = np.ones(n)
    
    # Linear coefficients for the equality constraints
    coef = np.zeros((1, n))
    coef[0, :] = labels

    # Solve quadratic programming problem, store result to alpha matrix as a
    # (Hessian, weights, eq_coeffs, eq_constants, ineq_coeffs, ineq_constants, lower_bounds, upper_bounds)
    a = solve_quadprog(hes, w, coef, np.zeros(1), None, None, np.zeros(n), 1.0)

    # Select alphas which have nonnegligible support
    vec_idx = a > 1e-6
    v = data[:, vec_idx]
    
    # Replace weight with nonnegligible support alphas. Same as labels
    w_fix = a[vec_idx]
    l = labels[vec_idx]

    # Find all alphas which are useful in the decision margin
    # Logical AND generates boolean array meets condition in parameter
    # That alphas indicates the useful one which has value TRUE
    marg_a = np.logical_and(a > 1e-6, a < 1.0 - 1e-6)

    # Compute bias
    if np.any(marg_a):
        # Bias is average value of transpose(label) - transpose(alpha * label) dot (gram matrix TRUE alpha part)
        b = np.mean(labels[marg_a].T - (a * labels).T.dot(g_mat[:, marg_a]))
    else:
        # No support vectors
        b = 0
        
    return v, w_fix, l, b

In [5]:
def svm_predict(data, w, b, v, l):
    '''
    Generate score of the prediction
    Use polynomial kernel to generate gram matrix and get scores
    @param data Dataset for binary SVM prediction
    @param w Weights
    @param b Bias
    @param v Support Vector
    @param l Labels correspond to Support Vector
    @return Scores of prediction
    '''
    # Gram matrix using polynomial kernel
    gram = (np.dot(data.T, v) + 1) ** 2
    # y = w x + b
    scores = gram.dot(w * l) + b
    return scores.ravel()

In [6]:
def relabel(num, sample_set):
    '''
    Convert label from 0-9 to +1 or -1
    @param num Choose label of data, value is 0-9, this one will be evaluated as +1
    @param sample_set Dataset to relabel
    @return Dataset in binary label form
    '''
    # Deep copy of original data
    binary = sample_set.copy(deep=True)
    
    # Remark labels
    binary.loc[binary['label'] != num, 'label'] = -1
    binary.loc[binary['label'] == num, 'label'] = 1
    return binary

In [7]:
def training(data):
    '''
    Use binary SVM to train recognizing and catogorizing the pixel to labels
    This function will get copy of original data and parse this data into label part and pixel part
    Then normalize the value in pixel part from 0-255 to 0-1
    Finally use the processed pixel data to obtain training score
    @param data Raw data which has label and pixels in same dataset
    @return Factors contribute to prediction, including weights, bias, support vectors and correspond label
    '''
    raw_set = data.copy(deep=True)
    # Obtain the label of raw data and transfer to numbers only
    raw_set_label = raw_set['label']
    raw_set_label = raw_set_label.values
    
    # Obtain the pixel data part of raw data and transfer to numbers only, then normalize
    raw_set_data = raw_set.drop(columns=['label'], inplace=True)
    raw_set_data = raw_set.values/255.0
    
    # Use the processed data to apply on binary SVM to get the score
    [vec, w, vec_lb, b] = bin_svm_train(raw_set_data.T, raw_set_label)
    return w, b, vec, vec_lb

In [8]:
def one_to_rest_train(train_data, sample_num):
    '''
    Use one to rest SVM to train
    This function will parse the training dataset and use one to rest SVM strategy
    Then return the factors for prediction use
    @param train_data: Training dataset
    @param sample_num: Number of sample from each digit subgroup
    @return Factors contribute to prediction, including weights, bias, support vectors and correspond label
    '''
    # Catagorize the dataset into subset 0-9
    group = train_data.groupby('label')

    # Store these dataset to each group
    zero = pd.DataFrame(group.get_group(0))
    one = pd.DataFrame(group.get_group(1))
    two = pd.DataFrame(group.get_group(2))
    three = pd.DataFrame(group.get_group(3))
    four = pd.DataFrame(group.get_group(4))
    five = pd.DataFrame(group.get_group(5))
    six = pd.DataFrame(group.get_group(6))
    seven = pd.DataFrame(group.get_group(7))
    eight = pd.DataFrame(group.get_group(8))
    nine = pd.DataFrame(group.get_group(9))

    # Obtain the pure pixel datasets
    zero_data = zero.drop(columns=['label'])
    one_data = one.drop(columns=['label'])
    two_data = two.drop(columns=['label'])
    three_data = three.drop(columns=['label'])
    four_data = four.drop(columns=['label'])
    five_data = five.drop(columns=['label'])
    six_data = six.drop(columns=['label'])
    seven_data = seven.drop(columns=['label'])
    eight_data = eight.drop(columns=['label'])
    nine_data = nine.drop(columns=['label'])

    # Set random sample size from each label group for training
    # WARNING: The ONE TO REST SVM requires a lot of time
    # Larger sample number may cause long time training
    count = sample_num

    # Random select
    rdm_zero = zero.sample(count)
    rdm_one = one.sample(count)
    rdm_two = two.sample(count)
    rdm_three = three.sample(count)
    rdm_four = four.sample(count)
    rdm_five = five.sample(count)
    rdm_six = six.sample(count)
    rdm_seven = seven.sample(count)
    rdm_eight = eight.sample(count)
    rdm_nine = nine.sample(count)

    # Concrete selected sample training data into whole dataset
    sample_set = rdm_zero
    sample_set = sample_set.append(rdm_one)
    sample_set = sample_set.append(rdm_two)
    sample_set = sample_set.append(rdm_three)
    sample_set = sample_set.append(rdm_four)
    sample_set = sample_set.append(rdm_five)
    sample_set = sample_set.append(rdm_six)
    sample_set = sample_set.append(rdm_seven)
    sample_set = sample_set.append(rdm_eight)
    sample_set = sample_set.append(rdm_nine)

    # Critical process of ONE TO REST SVM

    # Relabel the dataset to primary set with +1 and rest sets with -1
    # E.g. If we want to know the score of label 0 we need to label the "0" ones to +1 and other sets to be -1
    class_zero = relabel(0, sample_set)
    class_one = relabel(1, sample_set)
    class_two = relabel(2, sample_set)
    class_three = relabel(3, sample_set)
    class_four = relabel(4, sample_set)
    class_five = relabel(5, sample_set)
    class_six = relabel(6, sample_set)
    class_seven = relabel(7, sample_set)
    class_eight = relabel(8, sample_set)
    class_nine = relabel(9, sample_set)

    # Use binary SVM training to obtain four parts data for evaluating trends of each dataset
    w = [None] * 10 # weights, which is w in w x + b
    b = [None] * 10 # bias, which is b in w x + b
    v = [None] * 10 # support vectors, which is used for prediction
    l = [None] * 10 # labels correspond to support vectors, which is used for prediction
    [w[0], b[0], v[0], l[0]] = training(class_zero)
    [w[1], b[1], v[1], l[1]] = training(class_one)
    [w[2], b[2], v[2], l[2]] = training(class_two)
    [w[3], b[3], v[3], l[3]] = training(class_three)
    [w[4], b[4], v[4], l[4]] = training(class_four)
    [w[5], b[5], v[5], l[5]] = training(class_five)
    [w[6], b[6], v[6], l[6]] = training(class_six)
    [w[7], b[7], v[7], l[7]] = training(class_seven)
    [w[8], b[8], v[8], l[8]] = training(class_eight)
    [w[9], b[9], v[9], l[9]] = training(class_nine)
    return w, b, v, l

In [9]:
def one_to_rest_predict(test_data, w, b, v, l):
    '''
    Predict the test data and obtain number of correct prediction
    @param test_data Test dataset
    @param w Weights
    @param b Bias
    @param v Support Vector
    @param l Labels correspond to Support Vector
    @return Number of correct prediction
    '''
    # Obtain the raw data from test dataset
    testCut = test_data.copy(deep=True)

    # Obtain label part from raw data
    label = testCut['label']

    # Obtain pixel part from raw data
    testCut.drop(columns = ['label'], inplace=True)

    # Normalize the pixel data
    testCut = testCut.values/255.0

    # Use the training knowledge to predict test data
    correct = 0 # Count for correct prediction
    # Prediction for each row of data in the test dataset
    for j in range(testCut.shape[0]):
        piece = testCut[j] # Single row of data
        result = [None] * 10 # Score array to store the score of each label from 0-9
        # Store prediction evaluating scores to score array correspond to label 0-9
        for i in range(10):
            result[i] = svm_predict(piece, w[i], b[i], v[i], l[i])
        # The final prediction label should be the label which has highest score
        value = result.index(max(result))
        # Increment correct prediction count
        if (value == label.iloc[j]):
            correct = correct + 1

    return correct

In [10]:
def get_acc(sub, w, b, v, l):
    '''
    Get accuracy for data prediction
    @param sub Dataset, can be all data or subgroup of data
    @param w Weights
    @param b Bias
    @param v Support Vector
    @param l Labels correspond to Support Vector
    @return Accuracy of prediction
    '''
    correct_count = one_to_rest_predict(sub, w, b, v, l)
    accuracy = correct_count / sub.shape[0]
    return accuracy

In [11]:
# Use multiclass SVM to train the data
[w, b, v, l] = one_to_rest_train(trainData, sample_count)

# Parse test dataset to digits 0-9 group
group = testData.groupby('label')

# Obtain accuracy of prediction for each digits
for i in range(10):
    sub = pd.DataFrame(group.get_group(i))
    sub_accuracy= get_acc(sub, w, b, v, l)
    print("Accuracy of digit %d is: %.2f%%" % (i, sub_accuracy * 100))

# Obtain overall accuracy of prediction for all
correct_count = one_to_rest_predict(testData, w, b, v, l)
accuracy = correct_count / testData.shape[0]
print("Overall accuracy is: %.2f%%" % (accuracy * 100))

Accuracy of digit 0 is: 97.25%
Accuracy of digit 1 is: 97.53%
Accuracy of digit 2 is: 93.02%
Accuracy of digit 3 is: 87.84%
Accuracy of digit 4 is: 96.14%
Accuracy of digit 5 is: 94.20%
Accuracy of digit 6 is: 95.40%
Accuracy of digit 7 is: 93.64%
Accuracy of digit 8 is: 90.60%
Accuracy of digit 9 is: 87.66%
Overall accuracy is: 93.45%
